# SDC Ratio and SDC Impact

SDC ratio is a stable metrics. If fault injection campaign sample the same program many time and calculate the SDC ratio, for each campaign the SDC ratio will be the same.

SDC Impact is a unstable metrics. For each fault injection campaign program's expected SDC impact may change a lot.

In [24]:
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math

In [25]:
cg_df = pd.read_csv("cg_complete.csv")
#cg_df = pd.read_csv("fft_exhaust.csv")
highbit_dataset = cg_df[cg_df.bit > 32]
exp_dataset = cg_df[cg_df.bit > 51]
mantissa_dataset = cg_df[cg_df.bit < 52]

In [30]:
def calculateSDCFrequency(df, line):
    temp = df[df.Line == line]
    numberOfSDC = len(temp[temp.outcome == "SDC"])
    
    return numberOfSDC

def calculateSDCratio(df, line, rescale_ratio=1):
    temp = df[df.Line == line]
    numberOfFaultInjection = len(temp)
    numberOfSDC = len(temp[temp.outcome == "SDC"])

    if numberOfFaultInjection is not 0: 
        return float(numberOfSDC)/numberOfFaultInjection * rescale_ratio
    else:
        return 0
    
def calculateExpectedSDCImpact(df, line):
    temp = df[df.Line == line]
    temp = temp[temp.outcome == "SDC"]
    temp = temp[temp.diffnormr != float("inf")]
    
    if len(temp) == 0:
        return 0
    else:
        temp = temp.diffnormr.abs()/temp.out_xor.abs()
    
    return np.mean(temp)

def sampleFunction(dataset, sample_size):
    return dataset.sample(n=sample_size)

each_line_sdc_ratio_distribution_random = {}
each_line_sdc_ratio_distribution_hightbit = {}

each_line_sdc_fre_distribution_random = {}
each_line_sdc_fre_distribution_hightbit = {}

each_line_expected_sdc_impact_distribution_random = {}
each_line_expected_sdc_impact_distribution_highbit= {}

for i in range(1000):
    exp_sample = sampleFunction(exp_dataset, int(500 * 0.8))
    mantissa_sample = sampleFunction(mantissa_dataset, int(500 * 0.2))
    random_sample = sampleFunction(cg_df, 500)
    for line in cg_df.Line.unique():
        if line not in each_line_sdc_ratio_distribution_random:
            
            #SDC frequency
            each_line_sdc_fre_distribution_random[line] = []
            each_line_sdc_fre_distribution_hightbit[line] = []
            
            #SDC ratio
            each_line_sdc_ratio_distribution_random[line] = []
            each_line_sdc_ratio_distribution_hightbit[line] = []
            
            #SDC impact
            each_line_expected_sdc_impact_distribution_random[line] = []
            each_line_expected_sdc_impact_distribution_highbit[line] = []
            
        #SDC frequency
        each_line_sdc_fre_distribution_random[line].append(calculateSDCFrequency(random_sample, line))
        each_line_sdc_fre_distribution_random[line].append(calculateSDCFrequency(highbit_sample, line))
        
        #SDC ratio
        each_line_sdc_ratio_distribution_random[line].append(calculateSDCratio(random_sample, line))
        each_line_sdc_ratio_distribution_hightbit[line].append(calculateSDCratio(exp_sample, line) * 12.0/64.0 + calculateSDCratio(mantissa_sample, line) * 52.0/64.0)
        
        #SDC impact
        each_line_expected_sdc_impact_distribution_random[line].append(calculateExpectedSDCImpact(random_sample, line))
        each_line_expected_sdc_impact_distribution_highbit[line].append(calculateExpectedSDCImpact(highbit_sample, line))

In [31]:
#highbit_sample = sampleFunction(highbit_dataset, 10000)
#random_sample = sampleFunction(cg_df, 10000)
#print each_line_expected_sdc_impact_distribution_highbit
#print each_line_expected_sdc_impact_distribution_random
for line in cg_df.Line.unique():
    print line
    #SDC frequency
    #print "SDC Fre random", np.mean(each_line_sdc_fre_distribution_random[line]), np.var(each_line_sdc_fre_distribution_random[line])
    #print "SDC Fre highbit", np.mean(each_line_sdc_fre_distribution_random[line]), np.var(each_line_sdc_fre_distribution_random[line])
    
    #SDC Ratio
    print "SDC Ratio random", np.mean(each_line_sdc_ratio_distribution_random[line]), np.var(each_line_sdc_ratio_distribution_random[line])
    print "SDC Ratio highbit", np.mean(each_line_sdc_ratio_distribution_hightbit[line]), np.var(each_line_sdc_ratio_distribution_hightbit[line])
    print "SDC Ratio ", calculateSDCratio(cg_df, line)
    #SDC impact
    #print "SDC Impact random", np.mean(each_line_expected_sdc_impact_distribution_random[line]), np.var(each_line_expected_sdc_impact_distribution_random[line])
    #print "SDC Impact highbit", np.mean(each_line_expected_sdc_impact_distribution_highbit[line]), np.var(each_line_expected_sdc_impact_distribution_highbit[line])
    #print "SDC Impact ", calculateExpectedSDCImpact(cg_df, line)
    
#for line in cg_df.Line.unique():
#    print line
    #print "SDC Ratio ", calculateSDCratio(cg_df, line)
#    print "SDC Impact ", calculateExpectedSDCImpact(cg_df, line)

#for line in cg_df.Line.unique():
#    print line
#    print np.mean(each_line_expected_sdc_impact_distribution_random[line])
#    print np.mean(each_line_expected_sdc_impact_distribution_highbit[line])
#    print " "

167
SDC Ratio random 0.09461475851391093 0.002210770399172481
SDC Ratio highbit 0.08929361567736614 0.00028078690418279704
SDC Ratio  0.096435546875
175
SDC Ratio random 0.0799478465978466 0.021173498787336754
SDC Ratio highbit 0.07324431818181819 0.0028025809529885473
SDC Ratio  0.076171875
33
SDC Ratio random 0.08099334275210637 0.00521549925968701
SDC Ratio highbit 0.07551789979322149 0.0007891467201502904
SDC Ratio  0.0826822916667
25
SDC Ratio random 0.08914436025013975 0.00027455136914134737
SDC Ratio highbit 0.08218628835002406 3.680623290191644e-05
SDC Ratio  0.0889020647321
48
SDC Ratio random 0.1036961652400449 0.0030959231476732738
SDC Ratio highbit 0.09287728255310576 0.0007378282646623481
SDC Ratio  0.100260416667
75
SDC Ratio random 0.0765 0.060897749999999994
SDC Ratio highbit 0.063109375 0.011340693115234375
SDC Ratio  0.1875
76
SDC Ratio random 0.007 0.005284333333333331
SDC Ratio highbit 0.00546875 0.0008656005859375
SDC Ratio  0.015625
87
SDC Ratio random 0.009389754